In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("../.cache")

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("../files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(llm=llm,
                                  max_token_limit=200,
                                  return_messages=True,
                                  memory_key="chat_history")


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful assistant.
            Answer questions using only the following context.
            If you don't know the answer just say you don't know, don't make it up:
            \n\n{context}
            """,
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "chat_history": RunnableLambda(load_memory),
    }
    | prompt
    | llm
)

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result.content)

Created a chunk of size 717, which is longer than the specified 600
Created a chunk of size 608, which is longer than the specified 600
Created a chunk of size 642, which is longer than the specified 600
Created a chunk of size 1444, which is longer than the specified 600
Created a chunk of size 1251, which is longer than the specified 600
Created a chunk of size 1012, which is longer than the specified 600
Created a chunk of size 1493, which is longer than the specified 600
Created a chunk of size 819, which is longer than the specified 600
Created a chunk of size 1458, which is longer than the specified 600
Created a chunk of size 1411, which is longer than the specified 600
Created a chunk of size 742, which is longer than the specified 600
Created a chunk of size 669, which is longer than the specified 600
Created a chunk of size 906, which is longer than the specified 600
Created a chunk of size 703, which is longer than the specified 600
Created a chunk of size 1137, which is lon

In [2]:
questions = [
    "Is Aaronson guilty?",
    "What message did he write in the table?",
    "Who is Julia?",
]

for question in questions:
    invoke_chain(question)

Yes, according to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.
He wrote "FREEDOM IS SLAVERY" and "TWO AND TWO MAKE FIVE" on the table.
Julia is a character mentioned in the document who seems to have a significant connection with the main character.


In [3]:
load_memory(_)

[HumanMessage(content='Is Aaronson guilty?'),
 AIMessage(content='Yes, according to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'),
 HumanMessage(content='What message did he write in the table?'),
 AIMessage(content='He wrote "FREEDOM IS SLAVERY" and "TWO AND TWO MAKE FIVE" on the table.'),
 HumanMessage(content='Who is Julia?'),
 AIMessage(content='Julia is a character mentioned in the document who seems to have a significant connection with the main character.')]